In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter, OrderedDict
import pickle

In [3]:
import datasets

In [4]:
import glob

In [5]:
import json

In [6]:
from tqdm.notebook import tqdm

In [7]:
json_files = [
    json_file_id.split('/')[-1][:-5]
    for json_file_id in glob.glob('../data/json_files/*.json')
]


In [8]:
# the path to the JSON file
file_path = "../data/json_files/"

doc2json = {}

for doc in tqdm(json_files):
    # open the file in read mode
    file_path = "../data/json_files/" + doc + ".json"
    with open(file_path, "r", encoding='utf-8',) as file:
        # Load the JSON data
        doc2json[doc] = json.load(file)

  0%|          | 0/17486 [00:00<?, ?it/s]

In [9]:
len(doc2json)

17486

In [10]:

# read the metadat file into a pandas dataframe
metadata_file = '../data/Royal_Society_Corpus_open_v6.0_meta.tsv'

metadata_df = pd.read_csv(metadata_file, delimiter='\t')

metadata_df.head()

,id,issn,title,fpage,lpage,year,volume,journal,author,type,...,tokens,visualizationLink,doi,jstorLink,hasAbstract,isAbstractOf,primaryTopic,primaryTopicPercentage,secondaryTopic,secondaryTopicPercentage
0,rspa_1905_0001,0950-1207,"Address delivered by the President, Sir Willia...",1,29,1905,76,Proceedings of the Royal Society of London. Se...,"Sir William Huggins K. C. B., O. M., F. R. S.",speech,...,14279,http://corpora.clarin-d.uni-saarland.de/surpri...,10.1098/rspa.1905.0001,NaN,NaN,NaN,Biography,87.018807,Immunology,3.397728
1,rspa_1905_0002,0950-1207,"The boring of the Simplon Tunnel, and the dist...",29,33,1905,76,Proceedings of the Royal Society of London. Se...,"Francis Fox, M. Inst. C. E. |C. V. Boys, F. R. S.",article,...,1816,http://corpora.clarin-d.uni-saarland.de/surpri...,10.1098/rspa.1905.0002,NaN,NaN,NaN,Geography,34.843817,Thermodynamics,26.224835
2,rspa_1905_0003,0950-1207,On a method of finding the conductivity for heat.,34,48,1905,76,Proceedings of the Royal Society of London. Se...,"Professor C. Niven, F. R. S.",article,...,3624,http://corpora.clarin-d.uni-saarland.de/surpri...,10.1098/rspa.1905.0003,NaN,NaN,NaN,Electricity,22.229782,Formulae,19.054271
3,rspa_1905_0004,0950-1207,Theory of the reflection of light near the pol...,49,65,1905,76,Proceedings of the Royal Society of London. Se...,"Richard C. Maclaurin, M. A., LL. D.|Professor ...",article,...,2722,http://corpora.clarin-d.uni-saarland.de/surpri...,10.1098/rspa.1905.0004,NaN,NaN,NaN,Tables,38.543461,Fluid Dynamics,28.361855
4,rspa_1905_0005,0950-1207,On the relation between variations of atmosphe...,66,86,1905,76,Proceedings of the Royal Society of London. Se...,"Captain H. G. Lyons|Sir W. E. Garstin, G. C. M...",article,...,10079,http://corpora.clarin-d.uni-saarland.de/surpri...,10.1098/rspa.1905.0005,NaN,NaN,NaN,Meteorology,51.203697,Geography,22.031809


In [11]:
# add year, decade, and period to the dictionaries in doc2json

for doc in tqdm(doc2json):
    df_row = metadata_df.loc[metadata_df['id'] == doc]
    year = df_row['year'].values[0]
    decade = df_row['decade'].values[0]
    period = df_row['period'].values[0]

    doc2json[doc]['year'] = year
    doc2json[doc]['decade'] = decade
    doc2json[doc]['period'] = period

  0%|          | 0/17486 [00:00<?, ?it/s]

In [12]:
doc2json_list = list(doc2json.values())
len(doc2json_list)

17486

In [14]:
doc_dataset = datasets.Dataset.from_pandas(
    pd.DataFrame(data=doc2json_list)
)

In [15]:

# Push the doc_dataset to the HuggingFace hub
doc_dataset.push_to_hub("badrabdullah/royal_society_corpus_LLM_generated_metadata")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/badrabdullah/royal_society_corpus_LLM_generated_metadata/commit/ffb8003c44dfc2f33725b7be13bd6128dbe326be', commit_message='Upload dataset', commit_description='', oid='ffb8003c44dfc2f33725b7be13bd6128dbe326be', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
doc_dataset.save_to_disk('../data/royal_society_corpus_LLM_generated_metadata.hfds')

Saving the dataset (0/1 shards):   0%|          | 0/17486 [00:00<?, ? examples/s]